In [1]:
%load_ext autotime

import os, math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')
from tqdm import tqdm, trange

In [2]:
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Input, Conv1D, LSTM, MaxPooling1D
from sklearn.metrics import mean_squared_error, mean_absolute_error

def model(test_data, train_data):
    test_time = test_data['time']
    test_gl_value = test_data['gl_value']
    
    test_data.drop(columns = ['time'], inplace = True)
    train_data.rename(columns = {'bolus_type_normal dual':'bolus_type_normal_dual', 'bolus_type_square dual':'bolus_type_square_dual'}, inplace = True)
    train_data.drop(columns = ['time'], inplace = True)
    
    empty_train_col = [0]*len(train_data)
    for i, item in enumerate(test_data.columns):
        if item not in train_data.columns:
            train_data.insert(i, item, empty_train_col)

    empty_test_col = [0]*len(test_data)
    for i, item in enumerate(train_data.columns):
        if item not in test_data.columns:
            test_data.insert(i, item, empty_test_col)
    
    X_data = train_data.drop(columns = ['gl_predict'])
    y_data = train_data[['gl_predict']]
    input_dim = X_data.shape[1]
    
    scaler_x = MinMaxScaler()
    scaler_y = MinMaxScaler()
    scaler_x.fit(X_data)
    scaler_y.fit(y_data)
    scaled_X_train_data = scaler_x.transform(X_data)
    scaled_y_train_data = scaler_y.transform(y_data)
    scaled_X_train_data = np.reshape(scaled_X_train_data, (scaled_X_train_data.shape[0], 1, scaled_X_train_data.shape[1]))
    
    model = Sequential()
    model.add(Conv1D(filters = 32, kernel_size = 1, activation = 'relu', input_shape = (scaled_X_train_data.shape[1], scaled_X_train_data.shape[2])))
    model.add(LSTM(128, return_sequences = False))
    model.add(Dense(150, activation = 'relu'))
    model.add(Dropout(0.2))
    model.add(Dense(100, activation = 'relu'))
    model.add(Dropout(0.15))
    model.add(Dense(50, activation = 'relu'))
    model.add(Dense(16, activation = 'relu'))
    model.add(Dense(1, activation = 'relu'))
    model.compile(loss = 'mse', optimizer = 'adam')

    model.fit(scaled_X_train_data, scaled_y_train_data, epochs = 200, batch_size = 32, shuffle = False)
    
    X_test_data = test_data.drop(columns = ['gl_predict'])
    y_test_data = test_data[['gl_predict']]
    scaled_X_test_data = scaler_x.transform(X_test_data)
    scaled_X_test_data = np.reshape(scaled_X_test_data, (scaled_X_test_data.shape[0], 1, scaled_X_test_data.shape[1]))
    prediction = model.predict(scaled_X_test_data, batch_size = 32)
    scaled_prediction = scaler_y.inverse_transform(prediction)
    
    final_output = pd.DataFrame(columns = ['timestamp', 'predicted_BGL_value'])
    final_output['timestamp'] = test_time
#     final_output['true_BGL_value'] = test_gl_value
    final_output['predicted_BGL_value'] = scaled_prediction
    
    return (final_output, math.sqrt(mean_squared_error(scaled_prediction, y_test_data)), mean_absolute_error(scaled_prediction, y_test_data))

Using TensorFlow backend.


time: 4.89 s


In [3]:
test_data_list = ['540-ws-training.csv', '544-ws-training.csv', '552-ws-training.csv', '567-ws-training.csv', '584-ws-training.csv', '596-ws-training.csv']

rmse_list = {}
mae_list = {}
for file in os.listdir('../data/derived/5_min_complete_dataset/'):
    if file in test_data_list:
        train_data = pd.read_csv('../data/derived/5_min_complete_dataset/'+file)
        test_data = pd.read_csv('../data/derived_test/5_min_complete_dataset/'+file[0:7]+'testing.csv')
        final_output, rmse_error, mae_error = model(test_data, train_data)
        rmse_list[file] = rmse_error
        mae_list[file] = mae_error
        print ('{} - {}'.format(file, rmse_error))
        print ('{} - {}'.format(file, mae_error))
        final_output.to_csv('../data/output/conv_lstm/'+file[0:7]+'result_5min.csv')

Epoch 1/200
13247/13247 [==============================] - 7s 506us/step - loss: 0.0159
Epoch 2/200
13247/13247 [==============================] - 4s 326us/step - loss: 0.0027
Epoch 3/200
13247/13247 [==============================] - 4s 338us/step - loss: 0.0020
Epoch 4/200
13247/13247 [==============================] - 4s 294us/step - loss: 0.0026
Epoch 5/200
13247/13247 [==============================] - 4s 270us/step - loss: 0.0017
Epoch 6/200
13247/13247 [==============================] - 3s 262us/step - loss: 0.0015
Epoch 7/200
13247/13247 [==============================] - 4s 295us/step - loss: 0.0021
Epoch 8/200
13247/13247 [==============================] - 4s 320us/step - loss: 0.0016
Epoch 9/200
13247/13247 [==============================] - 4s 279us/step - loss: 0.0017
Epoch 10/200
13247/13247 [==============================] - 4s 297us/step - loss: 0.0031
Epoch 11/200
13247/13247 [==============================] - 4s 295us/step - loss: 0.0013
Epoch 12/200
13247/13247 [====

13247/13247 [==============================] - 3s 216us/step - loss: 7.7948e-04
Epoch 91/200
13247/13247 [==============================] - 3s 199us/step - loss: 9.4945e-04
Epoch 92/200
13247/13247 [==============================] - 3s 205us/step - loss: 8.3989e-04
Epoch 93/200
13247/13247 [==============================] - 3s 211us/step - loss: 7.1365e-04
Epoch 94/200
13247/13247 [==============================] - 3s 212us/step - loss: 7.3102e-04
Epoch 95/200
13247/13247 [==============================] - 3s 204us/step - loss: 7.1114e-04
Epoch 96/200
13247/13247 [==============================] - 3s 207us/step - loss: 7.9817e-04
Epoch 97/200
13247/13247 [==============================] - 3s 194us/step - loss: 7.2777e-04
Epoch 98/200
13247/13247 [==============================] - 3s 217us/step - loss: 8.3425e-04
Epoch 99/200
13247/13247 [==============================] - 3s 221us/step - loss: 7.3137e-04
Epoch 100/200
13247/13247 [==============================] - 3s 208us/step - loss: 

13247/13247 [==============================] - 2s 169us/step - loss: 6.7189e-04
Epoch 178/200
13247/13247 [==============================] - 2s 137us/step - loss: 6.7793e-04
Epoch 179/200
13247/13247 [==============================] - 2s 154us/step - loss: 6.5100e-04
Epoch 180/200
13247/13247 [==============================] - 2s 159us/step - loss: 7.0391e-04
Epoch 181/200
13247/13247 [==============================] - 2s 169us/step - loss: 6.5892e-04
Epoch 182/200
13247/13247 [==============================] - 2s 179us/step - loss: 6.7684e-04
Epoch 183/200
13247/13247 [==============================] - 2s 176us/step - loss: 6.6219e-04
Epoch 184/200
13247/13247 [==============================] - 2s 178us/step - loss: 6.9336e-04
Epoch 185/200
13247/13247 [==============================] - 2s 179us/step - loss: 6.5300e-04
Epoch 186/200
13247/13247 [==============================] - 2s 171us/step - loss: 6.3566e-04
Epoch 187/200
13247/13247 [==============================] - 2s 173us/step

13535/13535 [==============================] - 2s 182us/step - loss: 5.1038e-04
Epoch 65/200
13535/13535 [==============================] - 2s 171us/step - loss: 5.3979e-04
Epoch 66/200
13535/13535 [==============================] - 2s 182us/step - loss: 5.4357e-04
Epoch 67/200
13535/13535 [==============================] - 3s 188us/step - loss: 5.9687e-04
Epoch 68/200
13535/13535 [==============================] - 2s 179us/step - loss: 5.2942e-04
Epoch 69/200
13535/13535 [==============================] - 3s 187us/step - loss: 5.6411e-04
Epoch 70/200
13535/13535 [==============================] - 3s 186us/step - loss: 5.3542e-04
Epoch 71/200
13535/13535 [==============================] - 2s 181us/step - loss: 5.5244e-04
Epoch 72/200
13535/13535 [==============================] - 2s 179us/step - loss: 6.0685e-04
Epoch 73/200
13535/13535 [==============================] - 3s 185us/step - loss: 7.0487e-04
Epoch 74/200
13535/13535 [==============================] - 2s 179us/step - loss: 5

13535/13535 [==============================] - 2s 156us/step - loss: 4.5435e-04
Epoch 152/200
13535/13535 [==============================] - 2s 160us/step - loss: 5.6230e-04
Epoch 153/200
13535/13535 [==============================] - 2s 160us/step - loss: 5.0073e-04
Epoch 154/200
13535/13535 [==============================] - 2s 153us/step - loss: 4.7542e-04
Epoch 155/200
13535/13535 [==============================] - 2s 148us/step - loss: 4.8497e-04
Epoch 156/200
13535/13535 [==============================] - 2s 168us/step - loss: 4.6747e-04
Epoch 157/200
13535/13535 [==============================] - 2s 151us/step - loss: 4.8294e-04
Epoch 158/200
13535/13535 [==============================] - 2s 164us/step - loss: 5.1505e-04
Epoch 159/200
13535/13535 [==============================] - 2s 153us/step - loss: 5.1506e-04
Epoch 160/200
13535/13535 [==============================] - 2s 157us/step - loss: 5.0723e-04
Epoch 161/200
13535/13535 [==============================] - 2s 162us/step

...

11096/11096 [==============================] - 1s 131us/step - loss: 5.9774e-04
Epoch 190/200
11096/11096 [==============================] - 1s 132us/step - loss: 5.9993e-04
Epoch 191/200
11096/11096 [==============================] - 2s 136us/step - loss: 5.7354e-04
Epoch 192/200
11096/11096 [==============================] - 2s 137us/step - loss: 5.8549e-04
Epoch 193/200
11096/11096 [==============================] - 1s 130us/step - loss: 5.5609e-04
Epoch 194/200
11096/11096 [==============================] - 2s 138us/step - loss: 5.5476e-04
Epoch 195/200
11096/11096 [==============================] - 2s 141us/step - loss: 5.4670e-04
Epoch 196/200
11096/11096 [==============================] - 2s 136us/step - loss: 6.2237e-04
Epoch 197/200
11096/11096 [==============================] - 1s 125us/step - loss: 6.2003e-04
Epoch 198/200
11096/11096 [==============================] - 1s 125us/step - loss: 5.7485e-04
Epoch 199/200
11096/11096 [==============================] - 1s 118us/step

13104/13104 [==============================] - 2s 125us/step - loss: 3.4463e-04
Epoch 77/200
13104/13104 [==============================] - 2s 123us/step - loss: 2.9885e-04
Epoch 78/200
13104/13104 [==============================] - 1s 104us/step - loss: 2.7791e-04
Epoch 79/200
13104/13104 [==============================] - 1s 100us/step - loss: 3.5246e-04
Epoch 80/200
13104/13104 [==============================] - 2s 132us/step - loss: 4.0197e-04
Epoch 81/200
13104/13104 [==============================] - 2s 132us/step - loss: 2.6910e-04
Epoch 82/200
13104/13104 [==============================] - 2s 127us/step - loss: 3.4783e-04
Epoch 83/200
13104/13104 [==============================] - 2s 131us/step - loss: 2.8040e-04
Epoch 84/200
13104/13104 [==============================] - 2s 140us/step - loss: 2.7399e-04
Epoch 85/200
13104/13104 [==============================] - 2s 128us/step - loss: 2.8847e-04
Epoch 86/200
13104/13104 [==============================] - 2s 125us/step - loss: 2

13104/13104 [==============================] - 2s 128us/step - loss: 2.4624e-04
Epoch 164/200
13104/13104 [==============================] - 2s 130us/step - loss: 2.5235e-04
Epoch 165/200
13104/13104 [==============================] - 2s 127us/step - loss: 2.4325e-04
Epoch 166/200
13104/13104 [==============================] - 2s 141us/step - loss: 2.4063e-04
Epoch 167/200
13104/13104 [==============================] - 2s 131us/step - loss: 2.5455e-04
Epoch 168/200
13104/13104 [==============================] - 2s 125us/step - loss: 2.4982e-04
Epoch 169/200
13104/13104 [==============================] - 2s 121us/step - loss: 2.3551e-04
Epoch 170/200
13104/13104 [==============================] - 2s 126us/step - loss: 2.6498e-04
Epoch 171/200
13104/13104 [==============================] - 2s 128us/step - loss: 2.4753e-04
Epoch 172/200
13104/13104 [==============================] - 2s 136us/step - loss: 2.4885e-04
Epoch 173/200
13104/13104 [==============================] - 2s 129us/step

13104/13104 [==============================] - 1s 103us/step - loss: 5.5461e-04
Epoch 51/200
13104/13104 [==============================] - 1s 111us/step - loss: 5.2048e-04
Epoch 52/200
13104/13104 [==============================] - 1s 106us/step - loss: 5.0726e-04
Epoch 53/200
13104/13104 [==============================] - 1s 107us/step - loss: 4.7190e-04
Epoch 54/200
13104/13104 [==============================] - 1s 106us/step - loss: 4.8920e-04
Epoch 55/200
13104/13104 [==============================] - 2s 115us/step - loss: 5.3587e-04
Epoch 56/200
13104/13104 [==============================] - 2s 115us/step - loss: 4.9718e-04
Epoch 57/200
13104/13104 [==============================] - 1s 102us/step - loss: 5.4698e-04
Epoch 58/200
13104/13104 [==============================] - 1s 104us/step - loss: 5.6330e-04
Epoch 59/200
13104/13104 [==============================] - 1s 99us/step - loss: 5.4504e-04
Epoch 60/200
13104/13104 [==============================] - 1s 99us/step - loss: 5.0

Epoch 138/200
13104/13104 [==============================] - 1s 108us/step - loss: 4.1727e-04
Epoch 139/200
13104/13104 [==============================] - 1s 107us/step - loss: 4.4149e-04
Epoch 140/200
13104/13104 [==============================] - 1s 106us/step - loss: 4.2217e-04
Epoch 141/200
13104/13104 [==============================] - 1s 103us/step - loss: 4.1663e-04
Epoch 142/200
13104/13104 [==============================] - 1s 105us/step - loss: 4.1593e-04
Epoch 143/200
13104/13104 [==============================] - 1s 107us/step - loss: 4.3564e-04
Epoch 144/200
13104/13104 [==============================] - 1s 109us/step - loss: 4.1970e-04
Epoch 145/200
13104/13104 [==============================] - 1s 110us/step - loss: 4.2390e-04
Epoch 146/200
13104/13104 [==============================] - 1s 108us/step - loss: 4.0537e-04
Epoch 147/200
13104/13104 [==============================] - 1s 107us/step - loss: 4.1426e-04
Epoch 148/200
13104/13104 [==============================] -

In [4]:
rmse_list

{'584-ws-training.csv': 25.910773801241195,
 '567-ws-training.csv': 11.733225058294634,
 '596-ws-training.csv': 11.214986546475057,
 '552-ws-training.csv': 14.16040240048116,
 '544-ws-training.csv': 8.476112978134775,
 '540-ws-training.csv': 19.860239819487795}

time: 3.43 ms


In [5]:
mae_list

{'584-ws-training.csv': 23.804068481914186,
 '567-ws-training.csv': 7.793130782921555,
 '596-ws-training.csv': 7.31086794238818,
 '552-ws-training.csv': 12.931153303944754,
 '544-ws-training.csv': 6.225576419617381,
 '540-ws-training.csv': 18.226265817365288}

time: 7.33 ms


In [6]:
from statistics import pstdev, mean

rmse_values = rmse_list.values()
mae_values = mae_list.values()

print ('Mean - RMSE : {}'.format(mean(rmse_values)))
print ('Standard Deviation - RMSE : {}'.format(pstdev(rmse_values)))

print ('Mean - MAE : {}'.format(mean(mae_values)))
print ('Standard Deviation - MAE : {}'.format(pstdev(mae_values)))

Mean - RMSE : 15.225956767352436
Standard Deviation - RMSE : 5.924873397336718
Mean - MAE : 12.715177124691891
Standard Deviation - MAE : 6.4411472976554816
time: 7.89 ms
